<a href="https://colab.research.google.com/github/CSCCNY/final-project-recomposeclassics/blob/main/BaselineOnMidiFiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Recompose Classics : Baseline Model on Midi files**
######Hannah Do

---
***Summary***

1. Extract the notes and chords from the midi files with **music21**
2. Build Baseline Model
3. Run the models to predict and evaluate


---

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Dropout


#### 1. Extract the notes and chords from the midi files with **music21**

In [ ]:
def get_notes():

    notes = []

    for file in glob.glob("/content/gdrive/My Drive/RecomposeClassics/chopin/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('/content/gdrive/My Drive/RecomposeClassics/data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes


def prepare_sequences(notes, n_vocab):

    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)


    
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)


In [ ]:
notes = get_notes()

Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chp_op18.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p10.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p11.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chp_op31.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p1.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p12.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p14.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p15.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p13.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p18.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p16.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p17.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p2.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p19.mid
Parsing 

In [ ]:
n_vocab = len(set(notes))

network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
n_vocab

317

In [ ]:
notes

['B-4',
 'B-4',
 'B-4',
 'B-4',
 'B-4',
 'B-4',
 'B-4',
 'B-4',
 'B-4',
 'B-4',
 'B-4',
 'B-4',
 'B-4',
 'B-4',
 'B-4',
 'B-4',
 'B-2',
 'D5',
 '5.8.10',
 'E-5',
 '2.5',
 '5.8.10',
 'B-4',
 'E-3',
 'E-5',
 '3.7.10',
 'F5',
 '3.7',
 '3.7.10',
 'B-4',
 'D3',
 'F5',
 '5.8.10',
 'G5',
 '5.8',
 '5.8.10',
 '7.10.1',
 'E-3',
 '7.10.1',
 '7.10.1',
 'E3',
 '7.10.1',
 'B-5',
 '1.7',
 'F3',
 'C6',
 '5.8',
 'B-5',
 '8.0',
 'G#2',
 'G#5',
 'B4',
 'B-2',
 'B-5',
 '3.7',
 'G#5',
 '7.10',
 '3.7',
 'G5',
 'B-4',
 'B-2',
 'G#5',
 '2.5',
 'G5',
 '5.8',
 '10.2',
 'F5',
 'G#4',
 'E-3',
 'G5',
 '10.3',
 'F5',
 '3.7',
 '10.3',
 'B-4',
 'B-2',
 'D5',
 '5.8.10',
 'E-5',
 '2.5',
 '5.8.10',
 'B-4',
 'E-3',
 'E-5',
 '3.7.10',
 'F5',
 '3.7',
 '3.7.10',
 'B-4',
 'D3',
 'F5',
 '5.8.10',
 'G5',
 '5.8',
 '5.8.10',
 '7.10.1',
 'E-3',
 '7.10.1',
 '7.10',
 'C#5',
 'E3',
 '7.10',
 'C#5',
 'F3',
 'B-5',
 'C6',
 '5.8',
 'B-5',
 '8.0',
 'G#2',
 'B4',
 'B-2',
 'G#5',
 'B-5',
 '3.7',
 'G#5',
 '7.10',
 '3.7',
 'B-4',
 'B-2',
 '

In [ ]:
network_input

array([[[0.78233438],
        [0.78233438],
        [0.78233438],
        ...,
        [0.56466877],
        [0.55835962],
        [0.82334385]],

       [[0.78233438],
        [0.78233438],
        [0.78233438],
        ...,
        [0.55835962],
        [0.82334385],
        [0.90536278]],

       [[0.78233438],
        [0.78233438],
        [0.78233438],
        ...,
        [0.82334385],
        [0.90536278],
        [0.55835962]],

       ...,

       [[0.96845426],
        [0.88643533],
        [0.82018927],
        ...,
        [0.84227129],
        [0.88643533],
        [0.81388013]],

       [[0.88643533],
        [0.82018927],
        [0.97160883],
        ...,
        [0.88643533],
        [0.81388013],
        [0.96529968]],

       [[0.82018927],
        [0.97160883],
        [0.96845426],
        ...,
        [0.81388013],
        [0.96529968],
        [0.94637224]]])

In [ ]:
network_output

In [ ]:
len(network_input)

63329

In [ ]:

def baseline_network(network_input, n_vocab):

    model = Sequential()

    model.add(Dense(512, activation='relu'))


    model.add(Dense(512, input_shape=(network_input.shape(), n_vocab)))
    model.add(Dense(1024, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dense(512, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))

    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    return model


In [ ]:
baseline_model = baseline_network(network_input, n_vocab)

In [ ]:
def train(baseline_model, network_input, network_output):

    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    baseline_model.fit(network_input, network_output, epochs=50, batch_size=128, callbacks=callbacks_list)


In [ ]:
train(baseline_model, network_input, network_output)

In [ ]:
# all-in-one function

def train_network():

    notes = get_notes()

    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

###References
- Classical Piano Composer by Skuldur : https://github.com/Skuldur/Classical-Piano-Composer


